# MilvusとOpenAIを使用したフィルタ検索
### あなたの次の映画を見つける

このノートブックでは、OpenAIを使用して映画の説明の埋め込みを生成し、それらの埋め込みをMilvus内で使用して関連する映画を見つける方法について説明します。検索結果を絞り込み、新しいことを試すために、メタデータ検索を行うためのフィルタリングを使用します。この例のデータセットはHuggingFace datasetsから取得されており、8千を少し超える映画エントリが含まれています。

まず、このノートブックに必要なライブラリをダウンロードすることから始めましょう：
- `openai`はOpenAI埋め込みサービスとの通信に使用されます
- `pymilvus`はMilvusサーバーとの通信に使用されます
- `datasets`はデータセットのダウンロードに使用されます
- `tqdm`はプログレスバーに使用されます

In [ ]:
! pip install openai pymilvus datasets tqdm

必要なパッケージがインストールされたので、開始できます。まずMilvusサービスを起動しましょう。実行されるファイルは、このファイルのフォルダにある`docker-compose.yaml`です。このコマンドはMilvusスタンドアロンインスタンスを起動し、このテストで使用します。

In [4]:
! docker compose up -d

E0317 14:06:38.344884000 140704629352640 fork_posix.cc:76]             Other threads are currently calling into gRPC, skipping fork() handlers


[+] Running 1/0
 ⠿ Network milvus          Created                                         0.1s
 ⠋ Container milvus-etcd   Creating                                        0.0s
 ⠋ Container milvus-minio  Creating                                        0.0s
[+] Running 1/3
 ⠿ Network milvus          Created                                         0.1s
 ⠙ Container milvus-etcd   Creating                                        0.1s
 ⠙ Container milvus-minio  Creating                                        0.1s
[+] Running 2/3
 ⠿ Network milvus               Created                                    0.1s
 ⠿ Container milvus-etcd        Starting                                   0.2s
 ⠿ Container milvus-minio       Starting                                   0.2s
 ⠿ Container milvus-standalone  Created                                    0.1s
[+] Running 2/4
 ⠿ Network milvus               Created                                    0.1s
 ⠿ Container milvus-etcd        Starting                

Milvusが実行されている状態で、グローバル変数を設定できます：
- HOST: Milvusのホストアドレス
- PORT: Milvusのポート番号
- COLLECTION_NAME: Milvus内でコレクションに付ける名前
- DIMENSION: 埋め込みベクトルの次元数
- OPENAI_ENGINE: 使用する埋め込みモデル
- openai.api_key: あなたのOpenAIアカウントキー
- INDEX_PARAM: コレクションに使用するインデックス設定
- QUERY_PARAM: 使用する検索パラメータ
- BATCH_SIZE: 一度に埋め込みと挿入を行う映画の数

In [30]:
import openai

HOST = 'localhost'
PORT = 19530
COLLECTION_NAME = 'movie_search'
DIMENSION = 1536
OPENAI_ENGINE = 'text-embedding-3-small'
openai.api_key = 'sk-your_key'

INDEX_PARAM = {
    'metric_type':'L2',
    'index_type':"HNSW",
    'params':{'M': 8, 'efConstruction': 64}
}

QUERY_PARAM = {
    "metric_type": "L2",
    "params": {"ef": 64},
}

BATCH_SIZE = 1000

In [6]:
from pymilvus import connections, utility, FieldSchema, Collection, CollectionSchema, DataType

# Connect to Milvus Database
connections.connect(host=HOST, port=PORT)

In [18]:
# Remove collection if it already exists
if utility.has_collection(COLLECTION_NAME):
    utility.drop_collection(COLLECTION_NAME)

In [19]:
# Create collection which includes the id, title, and embedding.
fields = [
    FieldSchema(name='id', dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name='title', dtype=DataType.VARCHAR, max_length=64000),
    FieldSchema(name='type', dtype=DataType.VARCHAR, max_length=64000),
    FieldSchema(name='release_year', dtype=DataType.INT64),
    FieldSchema(name='rating', dtype=DataType.VARCHAR, max_length=64000),
    FieldSchema(name='description', dtype=DataType.VARCHAR, max_length=64000),
    FieldSchema(name='embedding', dtype=DataType.FLOAT_VECTOR, dim=DIMENSION)
]
schema = CollectionSchema(fields=fields)
collection = Collection(name=COLLECTION_NAME, schema=schema)

In [20]:
# Create the index on the collection and load it.
collection.create_index(field_name="embedding", index_params=INDEX_PARAM)
collection.load()

## データセット
Milvusが起動して実行されている状態で、データの取得を開始できます。Hugging Face Datasetsは多くの異なるユーザーデータセットを保持するハブであり、この例ではHuggingLearnersのnetflix-showsデータセットを使用します。このデータセットには8千本以上の映画とそのメタデータのペアが含まれています。各説明文を埋め込み、タイトル、タイプ、リリース年、評価と共にMilvus内に保存します。

In [21]:
import datasets

# Download the dataset 
dataset = datasets.load_dataset('hugginglearners/netflix-shows', split='train')

Found cached dataset csv (/Users/filiphaltmayer/.cache/huggingface/datasets/hugginglearners___csv/hugginglearners--netflix-shows-03475319fc65a05a/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


## データの挿入
マシンにデータが準備できたので、埋め込み処理を行ってMilvusに挿入を開始できます。埋め込み関数はテキストを受け取り、リスト形式で埋め込みベクトルを返します。

In [22]:
# Simple function that converts the texts to embeddings
def embed(texts):
    embeddings = openai.Embedding.create(
        input=texts,
        engine=OPENAI_ENGINE
    )
    return [x['embedding'] for x in embeddings['data']]


次のステップでは実際の挿入を行います。すべてのエントリを反復処理し、設定されたバッチサイズに達したらバッチを作成して挿入します。ループが終了した後、残りのバッチが存在する場合はそれを挿入します。

In [26]:
from tqdm import tqdm

data = [
    [], # title
    [], # type
    [], # release_year
    [], # rating
    [], # description
]

# Embed and insert in batches
for i in tqdm(range(0, len(dataset))):
    data[0].append(dataset[i]['title'] or '')
    data[1].append(dataset[i]['type'] or '')
    data[2].append(dataset[i]['release_year'] or -1)
    data[3].append(dataset[i]['rating'] or '')
    data[4].append(dataset[i]['description'] or '')
    if len(data[0]) % BATCH_SIZE == 0:
        data.append(embed(data[4]))
        collection.insert(data)
        data = [[],[],[],[],[]]

# Embed and insert the remainder 
if len(data[0]) != 0:
    data.append(embed(data[4]))
    collection.insert(data)
    data = [[],[],[],[],[]]


100%|██████████| 8807/8807 [00:31<00:00, 276.82it/s]


## データベースのクエリ
データがMilvusに安全に挿入されたので、クエリを実行できます。クエリは、検索したい映画の説明と使用するフィルターのタプルを受け取ります。フィルターの詳細については[こちら](https://milvus.io/docs/boolean.md)をご覧ください。検索では、まずあなたの説明とフィルター式が出力されます。その後、各結果について、結果の映画のスコア、タイトル、タイプ、公開年、評価、および説明が出力されます。

In [33]:
import textwrap

def query(query, top_k = 5):
    text, expr = query
    res = collection.search(embed(text), anns_field='embedding', expr = expr, param=QUERY_PARAM, limit = top_k, output_fields=['title', 'type', 'release_year', 'rating', 'description'])
    for i, hit in enumerate(res):
        print('Description:', text, 'Expression:', expr)
        print('Results:')
        for ii, hits in enumerate(hit):
            print('\t' + 'Rank:', ii + 1, 'Score:', hits.score, 'Title:', hits.entity.get('title'))
            print('\t\t' + 'Type:', hits.entity.get('type'), 'Release Year:', hits.entity.get('release_year'), 'Rating:', hits.entity.get('rating'))
            print(textwrap.fill(hits.entity.get('description'), 88))
            print()

my_query = ('movie about a fluffly animal', 'release_year < 2019 and rating like \"PG%\"')

query(my_query)

Description: movie about a fluffly animal Expression: release_year < 2019 and rating like "PG%"
Results:
	Rank: 1 Score: 0.30083978176116943 Title: The Lamb
		Type: Movie Release Year: 2017 Rating: PG
A big-dreaming donkey escapes his menial existence and befriends some free-spirited
animal pals in this imaginative retelling of the Nativity Story.

	Rank: 2 Score: 0.33528298139572144 Title: Puss in Boots
		Type: Movie Release Year: 2011 Rating: PG
The fabled feline heads to the Land of Giants with friends Humpty Dumpty and Kitty
Softpaws on a quest to nab its greatest treasure: the Golden Goose.

	Rank: 3 Score: 0.33528298139572144 Title: Puss in Boots
		Type: Movie Release Year: 2011 Rating: PG
The fabled feline heads to the Land of Giants with friends Humpty Dumpty and Kitty
Softpaws on a quest to nab its greatest treasure: the Golden Goose.

	Rank: 4 Score: 0.3414868116378784 Title: Show Dogs
		Type: Movie Release Year: 2018 Rating: PG
A rough and tough police dog must go undercover